In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options   # Http 헤더값 변경 목적 

# 로그인 관련 import 해옴. 참고한 블로그는 노션 인스타 로그인 문제 블로그 링크
from selenium.webdriver.common.by import By # HTML 요소를 찾아서 가져오고 싶을 때 쓰는 것
import time
import re
import pandas as pd
import random as rd

In [2]:
# 완성본.
# time.sleep(5) 기준 게시글 25개에 4분 소요.
# 4컴 돌려서 카테고리 6개 현재 2.5만인거 일단 1만개씩 돌리고 예비 모델 뽑자.
# 데이터는 4컴 돌리고 있는 만큼 오류 발생 시 데이터 손실이 크다. 10개 크롤링할때마다 pd.to_csv()로 꾸준히 파일로 저장해서 손실 최소화 하자.

In [50]:
# LinkDF = pd.read_csv("개인프로젝트_인스타 다중분류 모델용 링크 데이터.csv")
# LinkDF = LinkDF[["카테고리","태그","링크"]]
# LinkDF['게시자 아이디'] = [" " for n in range(len(LinkDF["링크"].unique()))]
# LinkDF['본문 내용'] = [" " for n in range(len(LinkDF["링크"].unique()))]
# LinkDF['해시태그'] = [" " for n in range(len(LinkDF["링크"].unique()))]

LinkDF = pd.read_csv("게시글 크롤링.csv")
LinkDF = LinkDF[["카테고리","태그","링크","게시자 아이디", "본문 내용","해시태그"]]

def insert_tag_row(idx, Oner_ID, MainText, hash_list):
    LinkDF.at[idx, '카테고리'] = LinkDF.iloc[idx]['카테고리']
    LinkDF.at[idx, '태그'] = LinkDF.iloc[idx]['태그']
    LinkDF.at[idx, '링크'] = LinkDF.iloc[idx]['링크']
    LinkDF.at[idx, '게시자 아이디'] = Oner_ID
    LinkDF.at[idx, '본문 내용'] = MainText
    LinkDF.at[idx, '해시태그'] = hash_list

In [2]:
options = Options()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

In [3]:
chrome_driver_path = "chromedriver.exe"  # 크롬 드라이버 설치 경로 입력
service = Service(chrome_driver_path)

# 웹 드라이버 실행
driver = webdriver.Chrome(options=options)
url ="https://www.instagram.com"
# url ="https://httpbin.org/headers"
# 여기서 로그인 하고 아래 본격적인 크롤링 시작하기
driver.set_window_position(100, 100)
driver.set_window_size(1024, 768)
driver.get(url)

In [53]:
# 각 요소들 경로 모음.

# 본문 찾기
MainText_path = [
    (By.CLASS_NAME, "_ap3a._aaco._aacu._aacx._aad7._aade"),
    (By.CLASS_NAME, "x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj"),
    (By.CSS_SELECTOR, "#mount_0_0_Nn > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6 > div > div:nth-child(1) > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > span > div > span"),
    (By.XPATH, "//*[@id='mount_0_0_Nn']/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div/span/div/span"),
    (By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div/span/div/span')
    ]

# 스크롤 가능한 요소 찾기
scroll_selectors = [
    (By.CLASS_NAME, "_a9z6._a9za"),
    (By.CLASS_NAME, "x5yr21d.xw2csxc.x1odjw0f.x1n2onr6"),
    (By.XPATH, '//*[@id="mount_0_0_IQ"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div[2]/div[1]/article/div/div[2]/div/div[2]/div[1]/ul'),
    (By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]'),
    (By.CSS_SELECTOR, '#mount_0_0_Nn > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6')
    ]

# 댓글창 전체를 리스트로 뽑아내는 경로
# 여기서 본문 + 일반 댓글의 게시글주인의 글 중 # 포함되어있는 것들 긁어오면 댐.
comments_path = [
    (By.CLASS_NAME, 'x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1'),
    (By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div[1]/div[1]/div/div[2]/div[1]/div[1]/div"),
    (By.CSS_SELECTOR, '#mount_0_0_Nn > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6 > div > div.x78zum5.xdt5ytf.x1iyjqo2 > div:nth-child(1) > div:nth-child(1) > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div:nth-child(1) > div'),
    (By.CSS_SELECTOR, '#mount_0_0_mD > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.xvkph5b.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x10o80wk.x14k21rp.xh8yej3.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6 > div > div:nth-child(3) > div > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div:nth-child(1) > div'),
    (By.XPATH, '//*[@id="mount_0_0_mD"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[3]/div/div/div[2]/div/div[1]/div')
    ]

# 본문 + 일반댓글에 없다면? 해당 경로를 통해 대댓글 열어서 가져가기.
comment_box_PATH = [
(By.CLASS_NAME, 'x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1'.replace(" ",".")),
(By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div[1]'),
(By.XPATH,'//*[@id="mount_0_0_D7"]/div/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div[1]'),
(By.CSS_SELECTOR,'#mount_0_0_D7 > div > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6 > div > div.x78zum5.xdt5ytf.x1iyjqo2 > div:nth-child(1)')
]

button_path = [
    (By.CLASS_NAME, 'x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x87ps6o.x1d5wrs8'),
    (By.CSS_SELECTOR, '#mount_0_0_AB > div > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div > div.x6s0dn4.x78zum5.xdt5ytf.xdj266r.xkrivgy.xat24cr.x1gryazu.x1n2onr6.xh8yej3 > div > div.x4h1yfo > div > div.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6 > div > div.x78zum5.xdt5ytf.x1iyjqo2 > div:nth-child(6) > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1.x540dpk > div'),
    (By.XPATH, '//*[@id="mount_0_0_AB"]/div/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div[6]/div[2]/div'),
    (By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div[6]/div[2]/div')
]

In [54]:
# 노트북 : 음식/요리 먼저.
target_tag = "음식/요리"
LinkDF[LinkDF['카테고리']==target_tag]
Link_list = LinkDF[(LinkDF['카테고리']==target_tag) & (LinkDF["게시자 아이디"] ==" ")].index.to_list()

In [48]:
# for link_idx in Link_list[:10]:
#     print(link_idx,":",LinkDF['링크'][link_idx])

In [55]:
# 완성!!!!!!!!!!!!!!!!!!!!!!!!! 
# 1202. 21:57 기준 일단 오류 발견된거 없음.
pre_save = 0
# for link in list(LinkDF['링크'][43637:53124]):  # 노트북 : 음식/요리 43124~66125 | 53124    일단 1만개만
for link_idx in Link_list:
    hash_list = []
    # link_idx = LinkDF[LinkDF['링크']== link].index.to_list()
    # link_idx = int(link_idx[0])
    link = LinkDF['링크'][link_idx]
    print(f"---------------------{link_idx}/{LinkDF['링크'].shape[0]}---------------------")
    driver.get(link) # 1차 로그인 화면
    time.sleep(rd.choice([3,3.3,3.5,3.7,4.0,4.1,4.3,4.5,4.7,5.0,5.1,5.2,5.3]))   # 계정 정지 방지용
    try: # id 크롤링
        Oner_ID = driver.find_element(By.CLASS_NAME,'x1i10hfl.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.xdl72j9.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.xjyslct.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x9f619.x1ypdohk.x1f6kntn.xwhw2v2.xl56j7k.x17ydfre.x2b8uid.xlyipyv.x87ps6o.x14atkfc.xcdnw81.x1i0vuye.xjbqb8w.xm3z3ea.x1x8b98j.x131883w.x16mih1h.x972fbf.xcfux6l.x1qhh985.xm0m39n.xt0psk2.xt7dq6l.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x1n5bzlp.xqnirrm.xj34u2y.x568u83')
        Oner_ID = Oner_ID.text
    except:
        Oner_ID = "None"
        LinkDF.at[link_idx, '게시자 아이디'] = "None"
        LinkDF.at[link_idx, '본문 내용'] = "None"
        LinkDF.at[link_idx, '해시태그'] = "None"
        continue    # 아이디도 안 긁혀오는 경우에는 페이지 삭제 확률이 높다.
    print("Oner_ID :", Oner_ID)
    time.sleep(rd.choice([0.3,0.5,0.7,1.0])) 

    # 본문내용
    for by, path in MainText_path:
        try:
            MainText = driver.find_element(by, path)
            break # 되면 더 찾지마
        except:               
            MainText = None
            continue
    if MainText is not None:
        MainText = MainText.text
    else:   # 본문 글이 없다면 다 None값 처리 
        LinkDF.at[link_idx, '게시자 아이디'] = "None"
        LinkDF.at[link_idx, '본문 내용'] = "None"
        LinkDF.at[link_idx, '해시태그'] = "None"  # 해시태그 중복 제거용
        continue
    # print("Before : MainText :", MainText)
    Maintext_list = MainText.split()
    for t in Maintext_list:
        if "#" in t:                # 본문에 해시태그가 있는 경우, 삭제 및 hash리스트에 빼서 넣어두기.
            MainText = MainText.replace(t,"")
            hash_list.append(t)
    time.sleep(rd.choice([0.3,0.5,0.7,1.0])) 
    MainText = re.sub('[^가-힣]+',' ', MainText) # 영어 및 이모티콘 같은 글 삭제
    if len(MainText) < 20:
        MainText = "None" # 전처리, 30자 이하라면 분석이 애매하니 삭제한다.
        insert_tag_row(link_idx, "None", "None", "None")  # 그리고 바로 추가 이후 삭제 처리 필요.
        continue
    else: 
        pass

    # 미리 스크롤 부분 찾아두기
    scrollable_div = None
    for by, selector in scroll_selectors:
        try:
            scrollable_div = driver.find_element(by, selector)
            break
        except:
            pass   

    # 본문에서 없으면 작동.
    # comment_box_PATH는 아래 박스 안에 있는거 찾는건디. 그럼 대댓찾는용이니께? 그냥 댓글이 빠졋네.
    idx = 0
    bad_try = 0
    for by, path in comments_path:
            try:
                comment = driver.find_elements(by, path)
                break
            except:
                pass
    Oner_text = ""
    for c in comment:   # 댓글에 있는 태그도 일단 검색해.
        if Oner_ID in c.text:
            Oner_text += c.text
    for text in Oner_text.split():  # 계정주 글 중에 # 포함되어있는 것들 추가.
        if "#" in text:
            for t in text.split("#")[1:]:
                t = "#"+t
                hash_list.append(t)
    
    time.sleep(rd.choice([0.5,0.7,1.0, 1.1,1.2,1.3]))              
    while (len(hash_list) < 1) and (bad_try<10):
        # 무언가 문제 발생 시 무한 스크롤하고 작동 정지하는 문제 있음.
        # bad_try 로 스크롤 10번만 하는 것으로 제한. 
        for by, path in comment_box_PATH:
            try:
                comment_box = driver.find_elements(by, path)
                break
            except:
                pass
        cocom = False
        # 댓글창
        for idx, box in enumerate(comment_box):
            # print(f"-----------------------------{idx}-----------------------------")
            if ("모두 보기" in box.text) and (len(box.text)>11) and (box.text.split()[0] == Oner_ID):    # 댓글 전체 + 그냥 답글 1개 모두 보기 이런식으로 또 2개 뜸  # 진짜 왜 .........?
                for by, path in button_path:
                    try:
                        box.find_element(by, path).click()
                        time.sleep(rd.choice([0.5,0.7,0.9])) 
                        driver.execute_script("arguments[0].scrollTop += arguments[1];", scrollable_div, 100)   # 스크롤 했으니 다시 크롤링 해야지.
                        time.sleep(rd.choice([1.2,1.5,1.7])) 
                        cocom = True
                    except:
                        pass
        # 대댓글 발견 시 작동.
        if cocom:
            for by, path in comment_box_PATH:
                try:
                    comment = driver.find_elements(by, path)
                    break
                except:
                    pass
            for idx, box in enumerate(comment):
                if box.text.split() != []:
                    # print(box.text.split())
                    if (box.text.split()[0] == Oner_ID):
                        hash_list = [text for text in box.text.split() if "#" in text]  # 일단 스크롤해서 더 밑에 깊게 묻혀있는게 아니면 갖고옴.
        if len(hash_list) < 1:  # 아니 뭐하느라 남들 댓글달때 지 태그를 안걸어놔 밑으로 계속 가
            driver.execute_script("arguments[0].scrollTop += arguments[1];", scrollable_div, 300)
            bad_try += 1
            time.sleep(rd.choice([1.0, 1.5,1.6,1.9]))
    # def로 했을때는 안되더니 그냥 따로 하니까 LinkDF에 들어가지는 중
    LinkDF.at[link_idx, '게시자 아이디'] = Oner_ID
    LinkDF.at[link_idx, '본문 내용'] = MainText
    LinkDF.at[link_idx, '해시태그'] = list(set(hash_list))  # 해시태그 중복 제거용

    if pre_save % 10 == 0:  # 10번째마다 저장
        LinkDF.to_csv("게시글 크롤링.csv")
    pre_save += 1
    time.sleep(rd.choice([0.5,0.7,1.0,1.2,1.5])) 

---------------------43140/145093---------------------
---------------------43186/145093---------------------


KeyboardInterrupt: 

In [ ]:
# # 백업용 // main_idx를 링크에 맞는 인덱스에 넣도록 수정 목적. (1202.20:24)
# for main_idx, link in enumerate(list(LinkDF['링크'][61845:61850])):
#     hash_list = []
#     print(f"---------------------{main_idx}/{LinkDF['링크'].shape[0]}---------------------")
#     driver.get(link) # 1차 로그인 화면 
#     time.sleep(3)

#     # id 크롤링
#     try:
#         time.sleep(1)
#         Oner_ID = driver.find_element(By.CLASS_NAME,'x1i10hfl.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.xdl72j9.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.xjyslct.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x9f619.x1ypdohk.x1f6kntn.xwhw2v2.xl56j7k.x17ydfre.x2b8uid.xlyipyv.x87ps6o.x14atkfc.xcdnw81.x1i0vuye.xjbqb8w.xm3z3ea.x1x8b98j.x131883w.x16mih1h.x972fbf.xcfux6l.x1qhh985.xm0m39n.xt0psk2.xt7dq6l.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x1n5bzlp.xqnirrm.xj34u2y.x568u83')
#         Oner_ID = Oner_ID.text
#     except:
#         Oner_ID = None
#         # print(f"{link[-10:]} 아이디 안댓음 도대체 왜 와이 어째서",end="\t")    # id의 댓글만 뽑아오기 용
#         continue    # 아이디도 안 긁혀오는 경우에는 페이지 삭제 확률이 높다.
#     print("Oner_ID :", Oner_ID)


#     # 본문내용
#     for by, path in MainText_path:
#         try:
#             MainText = driver.find_element(by, path)
#             break # 되면 더 찾지마
#         except:               
#             MainText = None
#             continue
#     MainText = MainText.text
#     # print("Before : MainText :", MainText)
#     Maintext_list = MainText.split()
#     for t in Maintext_list:
#         if "#" in t:                # 본문에 해시태그가 있는 경우, 삭제 및 hash리스트에 빼서 넣어두기.
#             MainText = MainText.replace(t,"")
#             hash_list.append(t)

#     MainText = re.sub('[^가-힣]+',' ', MainText) # 영어 및 이모티콘 같은 글 삭제
#     if len(MainText) < 20:
#         MainText = "20자 이하" # 전처리, 30자 이하라면 분석이 애매하니 삭제한다.
#         print("MainText 20자 이하")
#         insert_tag_row(main_idx, Oner_ID, MainText, hash_list)  # 그리고 바로 추가 이후 삭제 처리 필요.
#         continue
#     else: 
#         pass
#     print("MainText :", MainText)

#     # 미리 스크롤 부분 찾아두기
#     scrollable_div = None
#     for by, selector in scroll_selectors:
#         try:
#             scrollable_div = driver.find_element(by, selector)
#             break
#         except:
#             pass   
    
#     # 댓글, 대댓글 path // 본문도 같이 나옴 (???왜 ??? // 
#     # 아래 path로 하면 아이디 / 게시 시간 / 내용 <- 리스트로 나옴.
#     # 본문에서, MainText에서 태그가 많았다면 그냥 아래 while문은 안돌아가게 함.

#     # 본문에서 없으면 작동.
#     # comment_box_PATH는 아래 박스 안에 있는거 찾는건디. 그럼 대댓찾는용이니께? 그냥 댓글이 빠졋네.
#     idx = 0
#     bad_try = 0
#     for by, path in comments_path:
#             try:
#                 comment = driver.find_elements(by, path)
#                 break
#             except:
#                 pass
#     Oner_text = ""
#     for c in comment:   # 댓글에 있는 태그도 일단 검색해.
#         if Oner_ID in c.text:
#             Oner_text += c.text
#     for text in Oner_text.split():  # 계정주 글 중에 # 포함되어있는 것들 추가.
#         if "#" in text:
#             for t in text.split("#")[1:]:
#                 t = "#"+t
#                 hash_list.append(t)
    
               
#     while (len(hash_list) < 1) and (bad_try<10):
#         # 무언가 문제 발생 시 무한 스크롤하고 작동 정지하는 문제 있음.
#         # bad_try 로 스크롤 10번만 하는 것으로 제한. 
#         for by, path in comment_box_PATH:
#             try:
#                 comment_box = driver.find_elements(by, path)
#                 break
#             except:
#                 pass
#         cocom = False
#         # 댓글창
#         for idx, box in enumerate(comment_box):
#             # print(f"-----------------------------{idx}-----------------------------")
#             if ("모두 보기" in box.text) and (len(box.text)>11) and (box.text.split()[0] == Oner_ID):    # 댓글 전체 + 그냥 답글 1개 모두 보기 이런식으로 또 2개 뜸  # 진짜 왜 .........?
#                 for by, path in button_path:
#                     try:
#                         box.find_element(by, path).click()
#                         driver.execute_script("arguments[0].scrollTop += arguments[1];", scrollable_div, 100)   # 스크롤 했으니 다시 크롤링 해야지.
#                         time.sleep(2)
#                         cocom = True
#                     except:
#                         pass
#         # 대댓글 발견 시 작동.
#         if cocom:
#             for by, path in comment_box_PATH:
#                 try:
#                     comment = driver.find_elements(by, path)
#                     break
#                 except:
#                     pass
#             for idx, box in enumerate(comment):
#                 if box.text.split() != []:
#                     # print(box.text.split())
#                     if (box.text.split()[0] == Oner_ID):
#                         hash_list = [text for text in box.text.split() if "#" in text]  # 일단 스크롤해서 더 밑에 깊게 묻혀있는게 아니면 갖고옴.
#         if len(hash_list) < 1:  # 아니 뭐하느라 남들 댓글달때 지 태그를 안걸어놔 밑으로 계속 가
#             driver.execute_script("arguments[0].scrollTop += arguments[1];", scrollable_div, 300)
#             bad_try += 1
#             time.sleep(2)
#     print("hash_list :",hash_list)
#     # def로 했을때는 안되더니 그냥 따로 하니까 LinkDF에 들어가지는 중
#     LinkDF.at[main_idx, '게시자 아이디'] = Oner_ID
#     LinkDF.at[main_idx, '본문 내용'] = MainText
#     LinkDF.at[main_idx, '해시태그'] = list(set(hash_list))  # 해시태그 중복 제거용

#     # print(LinkDF)

In [50]:
pre_save = 15
pre_save % 10

5

In [48]:
# print(len(LinkDF['링크'].unique()))
# LinkDF['카테고리'].value_counts()
for category in LinkDF['카테고리'].unique().tolist():
    print(category, end=" : ")
    print(LinkDF[LinkDF['카테고리'] == category].index[0], end="~")
    print(LinkDF[LinkDF['카테고리'] == category].index[-1])


광고/홍보 : 0~18673
여행 : 18674~43123
음식/요리 : 43124~66125
일상 : 66126~91047
취미 : 91048~117907
패션/뷰티 : 117908~145092


In [ ]:
#정렬
# linkDF = pd.read_csv("개인프로젝트_인스타 다중분류 모델용 링크 데이터.csv")
# linkDF = linkDF[["카테고리","태그","링크"]]
# linkDF = linkDF.sort_values("카테고리")
# linkDF.reset_index(drop=True, inplace=True)
# linkDF.to_csv("개인프로젝트_인스타 다중분류 모델용 링크 데이터.csv")

In [45]:
LinkDF.sort_values("카테고리", inplace=True)
for category in LinkDF['카테고리'].unique().tolist():
    print(category)
    print(LinkDF[LinkDF['카테고리'] == category].index[0], end="~")
    print(LinkDF[LinkDF['카테고리'] == category].index[-1])

광고/홍보
114812~13032
여행
126849~55606
음식/요리
102870~105515
일상
5351~37426
취미
64325~47650
패션/뷰티
51097~97981
